In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
from indicnlp.tokenize import indic_tokenize
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
df_train = pd.read_csv('AWT_train.csv')
df_valid=pd.read_csv('AWT_dev.csv')
df_test=pd.read_csv('AWT_test_without_labels.csv')
df_train


,Text,Class
0,இதல்லம் ஒரு தீர்ப்பு நாட்டாமை தீர்ப்பை மற்று,Non-Abusive
1,"யாருடா அந்த கார்த்தி, நீ எங்கடா இருக்க ?",Non-Abusive
2,இரண்டு பேரின் (புண்டையை) சாமான்களை கோணிய ஊசால்...,Abusive
3,என்ன திமிர் இந்த பொண்ணுக்கு.....மக்களே இன்னும்...,Abusive
4,ஐயோ அந்த கார்த்திக் எ காட்டுங்க பா please,Non-Abusive
...,...,...
2785,ரெண்டுமே அவனா இல்ல அவளா....ஒரே கன்ஃப்யூசன்.......,Abusive
2786,ராஜேஷ்வரி செம்மயா இருக்கு ரொம்ப குளோசப்ல பார்க...,Abusive
2787,எனக்கு தெறிச்சு கார்த்தி னு ஒருத்தன் நல்லா ஓத்...,Abusive
2788,நீங்கள் கவலை கொல்லாதீர்கள் சகோதரி. சின்மை எவ்ள...,Abusive


In [4]:
df_valid

,Text,Class
0,சொத்துக்கே வழி இல்லாம இங்க கஸ்டப்பற்றாங்க இதுங...,Abusive
1,1:46 முட்டாள் ஆண்கள் இருக்குறவனுங்க மட்டும்தான...,Non-Abusive
2,சம்யுக்தா கிட்ட கேட்க வேண்டிய கேள்வி... proof...,Abusive
3,திவ்யா தேவிடியா நீ அசிங்கமா வீடியோ போட்டதே எல்...,Abusive
4,எதுக்கு இந்த பைத்தியத்துக்கு 2 episodes? பெரிய...,Non-Abusive
...,...,...
593,கஸ்த்தூரி ஒரு ஓலூ அப்படிதான் பேசுவால். ..,Abusive
594,Dr.Sharmi..... நீங்க என்ன சொல்லி இருக்கனும்......,Abusive
595,இந்த பொண்ணு பொய் சொல்வது சதவீதம் தெரியுது.காரண...,Abusive
596,விளங்கிடும் அடியேய் இதை விட கல்யாணம் பண்ணாதீங்கடி,Abusive


In [5]:
df_test

,id,Text
0,1,இவ ஒரு மானெங்கெட்ட பொறுக்கி. ஒரே ஒரு routine ஒ...
1,2,இப்டியே பேசிக்கிட்டே இருந்தா எப்டி..... யாரு ப...
2,3,அடக் கடவுளே இது என்னக் கொடுமையை ஊருல உலகத்துல ...
3,4,இதற்கு ஒரு தீர்வு இருக்கு. அவன் அவன் வேலை அவன...
4,5,தம்பி போய் நல்லவங்களை பேட்டிஎடு அவ சொல்வது அத்...
...,...,...
593,594,இது பைதியமா இல்லை நாம பைதியமா முடியல சாமி
594,595,இரண்டு வல்கர் சாக்கடை. கட்டி வச்சி அடிக்கணும்.
595,596,உன்கிட்ட தப்பிச்ச கார்த்தி சந்தோசமா இருக்கான்....
596,597,கார்த்தி அண்ட் திவ்யா ஒரு ஷோ பண்ணுங்க வெயிட்


In [7]:
def preprocess_text(text, lang='ta'): 
    normalizer = IndicNormalizerFactory().get_normalizer(lang)
    text = normalizer.normalize(text)
    tokens = list(indic_tokenize.trivial_tokenize(text, lang))
    return ' '.join(tokens).lower()

df_train['Text'] = df_train['Text'].apply(lambda x: preprocess_text(x, lang='ta'))
df_valid['Text'] = df_valid['Text'].apply(lambda x: preprocess_text(x, lang='ta'))
df_test['Text'] = df_test['Text'].apply(lambda x: preprocess_text(x, lang='ta'))


In [8]:
X_train=df_train['Text']
X_valid=df_valid['Text']
X_test=df_test['Text']
Y_train=df_train['Class']
Y_valid=df_valid['Class']



In [9]:
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_vec = vectorizer.fit_transform(X_train)
X_valid_vec = vectorizer.transform(X_valid)
X_test_vec=vectorizer.transform(X_test)

In [10]:
model=SVC(probability=True,C=10,kernel='rbf')
model.fit(X_train_vec, Y_train)

SVC(C=10, probability=True)

In [11]:
Y_valid_predicted=model.predict(X_valid_vec)


In [12]:
print("Classification Report:\n", classification_report(Y_valid, Y_valid_predicted))
print("Confusion Matrix:\n", confusion_matrix(Y_valid, Y_valid_predicted))
precision = precision_score(Y_valid, Y_valid_predicted, average='weighted')
recall = recall_score(Y_valid, Y_valid_predicted, average='weighted')
f1 = f1_score(Y_valid, Y_valid_predicted, average='weighted')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

Classification Report:
               precision    recall  f1-score   support

     Abusive       0.65      0.69      0.67       278
 Non-Abusive       0.71      0.67      0.69       320

    accuracy                           0.68       598
   macro avg       0.68      0.68      0.68       598
weighted avg       0.68      0.68      0.68       598

Confusion Matrix:
 [[192  86]
 [105 215]]
Precision: 0.68
Recall: 0.68
F1-Score: 0.68


In [13]:
Y_predicted=model.predict(X_test_vec)

In [14]:
Y_predicted

array(['Abusive', 'Abusive', 'Non-Abusive', 'Abusive', 'Abusive',
       'Abusive', 'Abusive', 'Abusive', 'Abusive', 'Non-Abusive',
       'Abusive', 'Abusive', 'Abusive', 'Abusive', 'Abusive', 'Abusive',
       'Non-Abusive', 'Abusive', 'Non-Abusive', 'Non-Abusive', 'Abusive',
       'Abusive', 'Non-Abusive', 'Non-Abusive', 'Abusive', 'Abusive',
       'Non-Abusive', 'Non-Abusive', 'Non-Abusive', 'Non-Abusive',
       'Abusive', 'Abusive', 'Abusive', 'Non-Abusive', 'Non-Abusive',
       'Abusive', 'Abusive', 'Abusive', 'Non-Abusive', 'Non-Abusive',
       'Non-Abusive', 'Non-Abusive', 'Non-Abusive', 'Abusive', 'Abusive',
       'Abusive', 'Abusive', 'Non-Abusive', 'Abusive', 'Abusive',
       'Abusive', 'Non-Abusive', 'Non-Abusive', 'Abusive', 'Abusive',
       'Abusive', 'Non-Abusive', 'Abusive', 'Abusive', 'Non-Abusive',
       'Non-Abusive', 'Abusive', 'Non-Abusive', 'Non-Abusive',
       'Non-Abusive', 'Non-Abusive', 'Abusive', 'Non-Abusive', 'Abusive',
       'Non-Abusive', 'Non-

In [15]:
import joblib


joblib.dump(model, 'tamil_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer_tamil.pkl')

print("Model saved successfully as 'tamil_model.pkl'")
print("Vectorizer saved successfully as 'tfidf_vectorizer_tamil.pkl'")


Model saved successfully as 'tamil_model.pkl'
Vectorizer saved successfully as 'tfidf_vectorizer_tamil.pkl'


In [17]:
predicted=pd.read_csv('Syndicate_IIITK_Tamil.csv')
predicted

,id,Class
0,1,Abusive
1,2,Non-Abusive
2,3,Non-Abusive
3,4,Non-Abusive
4,5,Abusive
...,...,...
593,594,Non-Abusive
594,595,Abusive
595,596,Abusive
596,597,Non-Abusive
